In [4]:
# !pip install transformers datasets tensorflow

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from datasets import load_dataset

In [6]:
# Load Twitter Sentiment dataset from Hugging Face
dataset = load_dataset("cardiffnlp/tweet_eval", "sentiment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/901k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/167k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
# Convert dataset to Pandas DataFrame
def dataset_to_dataframe(split):
    return pd.DataFrame({
        'text': dataset[split]['text'],
        'label': dataset[split]['label']
    })

In [8]:

df_train = dataset_to_dataframe("train")
df_test = dataset_to_dataframe("test")


In [9]:
# Load BERT tokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
def tokenize_data(texts, labels, max_length=128):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length)
    return {
        'input_ids': np.array(encodings['input_ids']),
        'attention_mask': np.array(encodings['attention_mask']),
        'labels': np.array(labels)
    }

In [11]:
# Tokenize dataset
train_data = tokenize_data(df_train['text'], df_train['label'])
test_data = tokenize_data(df_test['text'], df_test['label'])


In [12]:
# Tokenize dataset
train_data = tokenize_data(df_train['text'], df_train['label'])
test_data = tokenize_data(df_test['text'], df_test['label'])

# Load pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')


# Train the model
model.fit(
    x={'input_ids': train_data['input_ids'], 'attention_mask': train_data['attention_mask']},
    y=train_data['labels'],
    validation_data=({'input_ids': test_data['input_ids'], 'attention_mask': test_data['attention_mask']}, test_data['labels']),
    epochs=3,
    batch_size=8
)

# Save the trained model
model.save_pretrained("twitter_sentiment_model")
tokenizer.save_pretrained("twitter_sentiment_model")


Epoch 1/3
5702/5702 [==============================] - 1273s 216ms/step - loss: 0.6600 - accuracy: 0.6974 - val_loss: 0.6812 - val_accuracy: 0.6909
Epoch 2/3
5702/5702 [==============================] - 1229s 216ms/step - loss: 0.4644 - accuracy: 0.7998 - val_loss: 0.7687 - val_accuracy: 0.6888
Epoch 3/3
5702/5702 [==============================] - 1231s 216ms/step - loss: 0.2776 - accuracy: 0.8895 - val_loss: 0.9208 - val_accuracy: 0.6752


('twitter_sentiment_model/tokenizer_config.json',
 'twitter_sentiment_model/special_tokens_map.json',
 'twitter_sentiment_model/vocab.txt',
 'twitter_sentiment_model/added_tokens.json')

In [14]:
# Inference function
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)
    outputs = model(inputs)[0]
    prediction = tf.nn.softmax(outputs, axis=1)
    labels = ["Negative", "Neutral", "Positive"]
    return labels[np.argmax(prediction)]

# Example usage
example_tweet = "That's a good movie"
print("Predicted Sentiment:", predict_sentiment(example_tweet))

Predicted Sentiment: Positive
